# Vehicle Search Agent API Testing

Interactive notebook for testing the API endpoints.

**Prerequisites:** Make sure the API server is running:
```bash
python api/server.py
api documentation: http://localhost:8000/docs
```

In [ ]:
import requests
import json
from pprint import pprint

BASE_URL = "http://localhost:8000"

## 1. Health Check

In [ ]:
response = requests.get(f"{BASE_URL}/")
print(f"Status: {response.status_code}")
pprint(response.json())

## 2. Start a Conversation

In [ ]:
# Send first message
response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "I want a Jeep around $30k in Colorado"
    }
)

data = response.json()
session_id = data.get('session_id')

print(f"Session ID: {session_id}")
print(f"Mode: {data.get('mode')}")
print(f"\nAgent Response:")
print(data.get('response'))
print(f"\nVehicles Found: {len(data.get('vehicles', []))}")

## 3. View Filters

In [ ]:
print("Current Filters:")
pprint(data.get('filters'))

print("\nInferred Preferences:")
pprint(data.get('preferences'))

## 4. View Vehicle Listings

In [ ]:
vehicles = data.get('vehicles', [])
for i, vehicle in enumerate(vehicles[:5], 1):  # Show top 5
    v = vehicle.get('vehicle', {})
    listing = vehicle.get('retailListing', {})
    history = vehicle.get('history', {})
    photos = vehicle.get('photos')

    print(f"\n{'='*80}")
    print(f"#{i}. {v.get('year')} {v.get('make')} {v.get('model')} {v.get('trim', '')}")
    print(f"{'='*80}")

    # Price and basic info
    print(f"💰 Price:        ${listing.get('price', 0):,}")
    print(f"📏 Mileage:      {listing.get('miles', 0):,} miles")
    print(f"📍 Location:     {listing.get('city')}, {listing.get('state')} {listing.get('zip')}")
    print(f"🏢 Dealer:       {listing.get('dealer')}")

    # Vehicle details
    print(f"\n🚗 Vehicle Details:")
    print(f"   • Body Style:    {v.get('bodyStyle', 'N/A')}")
    print(f"   • Drivetrain:    {v.get('drivetrain', 'N/A')}")
    print(f"   • Engine:        {v.get('engine', 'N/A')}")
    print(f"   • Transmission:  {v.get('transmission', 'N/A')}")
    print(f"   • Exterior:      {v.get('exteriorColor', 'N/A')}")
    print(f"   • Interior:      {v.get('interiorColor', 'N/A')}")
    print(f"   • Doors:         {v.get('doors', 'N/A')}")
    print(f"   • Seats:         {v.get('seats', 'N/A')}")

    # History
    print(f"\n📋 Vehicle History:")
    print(f"   • Accidents:     {'Yes (' + str(history.get('accidentCount', 0)) + ')' if
history.get('accidents') else 'No'}")
    print(f"   • Owners:        {history.get('ownerCount', 'N/A')}")
    print(f"   • Usage:         {history.get('usageType', 'N/A')}")

    # Photos
    if photos and photos.get('retail'):
        photo_urls = photos.get('retail', [])
        print(f"\n📸 Photos:       {len(photo_urls)} available (cached)")
        print(f"   • First photo:   {photo_urls[0][:60]}...")
    else:
        print(f"\n📸 Photos:       No photos available")

    # Links
    print(f"\n🔗 Links:")
    print(f"   • VIN:           {v.get('vin')}")
    print(f"   • Listing:       {listing.get('vdp', 'N/A')[:70]}...")
    if listing.get('carfaxUrl'):
        print(f"   • CarFax:        {listing.get('carfaxUrl')[:70]}...")

print(f"\n{'='*80}")
print(f"Total vehicles: {len(vehicles)}")

## 5. Continue Conversation (Analytical Mode)

In [ ]:
# Ask about a specific vehicle
response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "Tell me more about #1",
        "session_id": session_id
    }
)

data = response.json()

print(f"Mode: {data.get('mode')}")
print(f"\nAgent Response:")
print(data.get('response'))

## 6. Get Session State

In [ ]:
response = requests.get(f"{BASE_URL}/session/{session_id}")
session_data = response.json()

print("Session State:")
print(f"  Filters: {session_data.get('filters')}")
print(f"  Preferences: {session_data.get('preferences')}")
print(f"  Vehicles: {len(session_data.get('vehicles', []))}")
print(f"  Conversation Length: {len(session_data.get('conversation_history', []))} messages")

## 7. View Conversation History

In [ ]:
print("Conversation History:")
print("=" * 70)

for msg in session_data.get('conversation_history', []):
    role = msg.get('role', 'unknown').upper()
    content = msg.get('content', '')
    
    if role == 'USER':
        print(f"\n👤 User: {content}")
    else:
        print(f"\n🤖 Agent: {content[:200]}...")  # Truncate long responses
    print("-" * 70)

## 8. Get Vehicle Details by VIN

In [ ]:
# Get VIN from first vehicle and test Auto.dev API directly
import os
from dotenv import load_dotenv

load_dotenv()

if vehicles:
    vin = vehicles[0].get('vehicle', {}).get('vin')
    
    if vin:
        print(f"Testing Auto.dev API for VIN: {vin}\n")
        print("=" * 70)
        
        # Get Auto.dev API key
        autodev_api_key = os.getenv('AUTODEV_API_KEY')
        
        if not autodev_api_key:
            print("❌ AUTODEV_API_KEY not found in environment")
        else:
            headers = {
                "Authorization": f"Bearer {autodev_api_key}",
                "Content-Type": "application/json"
            }
            
            # Get vehicle listing details
            print("\n1. Vehicle Listing Details:")
            listing_response = requests.get(
                f"https://api.auto.dev/listings/{vin}",
                headers=headers
            )
            
            if listing_response.status_code == 200:
                listing_data = listing_response.json()
                print("✅ Listing found")
                pprint(listing_data)
            else:
                print(f"❌ Error {listing_response.status_code}: {listing_response.text}")
            
            # Get vehicle photos
            print("\n2. Vehicle Photos:")
            photos_response = requests.get(
                f"https://api.auto.dev/photos/{vin}",
                headers=headers
            )
            
            if photos_response.status_code == 200:
                photos_data = photos_response.json()
                retail_photos = photos_data.get('data', {}).get('retail', [])
                
                print(f"✅ Found {len(retail_photos)} photos")
                
                if retail_photos:
                    print("\nFirst 5 photo URLs:")
                    for i, url in enumerate(retail_photos[:5], 1):
                        print(f"  {i}. {url}")
            else:
                print(f"❌ Error {photos_response.status_code}: {photos_response.text}")
    else:
        print("No VIN found in first vehicle")
else:
    print("No vehicles available to query")

## 9. Custom Message (Try Your Own!)

In [ ]:
# Try your own message
custom_message = "What's the safety rating of #1?"  

response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": custom_message,
        "session_id": session_id
    }
)

data = response.json()
print(f"Your Message: {custom_message}")
print(f"\nAgent Response:")
print(data.get('response'))

## 10. Reset Session

In [ ]:
response = requests.post(
    f"{BASE_URL}/session/reset",
    json={"session_id": session_id}
)

print("Session reset:")
pprint(response.json())

## 11. List All Active Sessions

In [ ]:
response = requests.get(f"{BASE_URL}/sessions")
sessions_data = response.json()

print(f"Active Sessions: {sessions_data.get('active_sessions')}")
print(f"Session IDs: {sessions_data.get('session_ids')}")